# Validação Cruzada

## Descrição

Use os dados do [Breast Cancer Dataset](https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Coimbra) UCI Machine Learning Repository.  

Discuta se é preciso normalizar os dados ou não. Se for preciso, faça a normalizaçao.  
 
Você vai verificar qual dentre os algoritmos abaixo é o melhor para classificar os dados. Utilize 20% dos dados para teste estratificando pela classe.

- Decision Tree  (criterion{“gini”, “entropy”, “log_loss”}, max_depth de 10 a 100 em multiplos de 10)
- K vizinhos (K = 1,3,5,11,21,31)  

Verifique que algoritmo tem a melhor acuracia media. Vamos usar o desenho experimental discutido em classe, de um k-fold para medir a acuracia media e um k-fold interno para escolher os hiperparâmetros (para cada conjunto de folds de treino). O algoritmo experimental esta descrito abaixo. Use 5-fold tanto para a avaliacao externa (para calcular a acuracia media) quanto para o loop interno (para escolher os hiperparametros).  

Obtenha o melhor conjunto de hiperparametros, retreine o modelo usando todos os dados de treino e avalie o desempenho com os dados de teste.

## Preparando os dados

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [19]:
dados = pd.read_csv('dataR2.csv')

dados

X = dados.iloc[:, : -1]
y = dados['Classification']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

normalizacao = StandardScaler()
normalizacao.fit(X_train)

X_train_std = pd.DataFrame(normalizacao.transform(X_train), columns=X_train.columns.tolist())
X_test_std = pd.DataFrame(normalizacao.transform(X_test), columns=X_train.columns.tolist())

## Importações Gerais

In [87]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

## Decision Tree

In [47]:
from sklearn.tree import DecisionTreeClassifier

In [67]:
criterion = ['gini', 'entropy', 'log_loss']
maxDepth = range(10, 101, 10)

maxAcc = float('-inf')
bestCrit= criterion[0]
bestDepth = maxDepth[0]

for cri in criterion:
    for maxD in maxDepth:

        clf = DecisionTreeClassifier(criterion=cri, max_depth=maxD, random_state=42)
        kf = KFold(n_splits=5)

        for f, (train_index, val_index) in enumerate(kf.split(X_train_std)):
            clf.fit(X_train_std.iloc[train_index], y_train.iloc[train_index])
            y_pred = clf.predict(X_train_std.iloc[val_index])
            
            acc = accuracy_score(y_train.iloc[val_index], y_pred)
            
            if maxAcc < acc:
                maxAcc = acc
                bestCrit = cri
                bestDepth = maxD


print("Accuracy: {0:.2f}%".format(maxAcc * 100))
print(f"Criterion: {bestCrit}")
print(f"Max depth: {bestDepth}")


Accuracy: 83.33%
Criterion: entropy
Max depth: 10


In [95]:
clf = DecisionTreeClassifier(criterion=bestCrit, max_depth=bestDepth, random_state=42)
clf.fit(X_train_std, y_train)

kf = KFold(n_splits=5)

acc = []

for f, (_, test_index) in enumerate(kf.split(X_test_std)):
    y_pred = clf.predict(X_test_std.iloc[test_index])
    acc.append(accuracy_score(y_test.iloc[test_index], y_pred))
    
print("Average accuracy: {0:.2f}%".format((sum(acc)/5) * 100))


Average accuracy: 75.00%


## KNeighborsClassifier (KNN)

In [53]:
from sklearn.neighbors import KNeighborsClassifier

In [84]:
kVizinhos = [1, 3, 5, 11, 21, 31]

maxAcc = float('-inf')
bestK = kVizinhos[0]

for k in kVizinhos:

    neigh = KNeighborsClassifier(n_neighbors=k)
    kf = KFold(n_splits=5)

    for f, (train_index, val_index) in enumerate(kf.split(X_train_std)):
        neigh.fit(X_train_std.iloc[train_index], y_train.iloc[train_index])
        y_pred = neigh.predict(X_train_std.iloc[val_index])
        
        acc = accuracy_score(y_train.iloc[val_index], y_pred)

        if maxAcc < acc:
            maxAcc = acc
            bestK = k

print("Accuracy: {0:.2f}%".format(maxAcc * 100))
print(f"K: {bestK}")

Accuracy: 94.74%
K: 3


In [92]:
neigh = KNeighborsClassifier(n_neighbors=bestK)
neigh.fit(X_train_std, y_train)

kf = KFold(n_splits=5)

acc = []

for f, (_, test_index) in enumerate(kf.split(X_test_std)):
    y_pred = neigh.predict(X_test_std.iloc[test_index])
    acc.append(accuracy_score(y_test.iloc[test_index], y_pred))
    
print("Average accuracy: {0:.2f}%".format((sum(acc)/5) * 100))

Average accuracy: 72.00%


## Comparação dos resultados

|Modelo|Acurácia (melhores hiperparâmetros)|Hiperparâmetros|Acurácia média|
|------|-----------------------------------|---------------|--------------|
|Decision Tree|83.33%|Criterion: entropy; Max depth: 10|75%|
|KNN|94.74%|K-vizinhos: 3|72%| 
